In [1]:
!pip install tika==1.19

In [4]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
#Import warnings filter
from warnings import simplefilter
#Ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [5]:
from tika import parser
import pandas as pd
import numpy as np

raw = parser.from_file('GC Funds 2008.pdf')
print(raw['content'])














































General Fund Revenues Proposed General Fund revenues and appropriated fund balance for FY 2008-09 

total $586,115,142. This is 6.7% ($36.85 million) higher than the budget approved for FY 2007-08. 

Property Tax revenues will account for 54% of the total revenue budget next fiscal year. The next largest 

sources of funds for general county operations are Federal and State Funds (17%) and Sales Tax 

revenues (13%). The FY 2008-09 adopted budget includes a general county wide tax rate of $0.7374 for 

each $100 of assessed property valuation. This is an increase of 4.6 cents over the tax rate during FY 

2008-09. Property Tax revenues, including payments and penalties for taxes due in prior years, will 

increase $29.2 million in FY 2008-09. In addition, the County expects an increase in Federal and State 

funds (+$11.9 million) and revenues derived from User Charges/Fees (+$4.6 million). Recent state 

legislation tied to Medicaid relief 

In [6]:
test = list(raw.values())

In [7]:
import re as re

In [8]:
makeitastring = ''.join(map(str, test))

In [9]:
type(makeitastring)

str

In [10]:
test1 = re.split('(?<!\d)[.]|[.](?!\d)',makeitastring )

In [11]:
df = pd.DataFrame({'text':test1})

In [12]:
df.head()

,text
0,200\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
1,This is 6.7% ($36.85 million) higher than the...
2,\n\nProperty Tax revenues will account for 54...
3,The next largest \n\nsources of funds for gen...
4,The FY 2008-09 adopted budget includes a gene...


In [13]:
df.dropna(subset=['text'], inplace=True)

In [14]:
df['text'] = df['text'].str.replace("[^a-zA-Z#]", " ")

In [15]:
df['text']

0                                                    ...
1       This is              million  higher than the...
2         Property Tax revenues will account for     ...
3       The next largest   sources of funds for gener...
4       The FY         adopted budget includes a gene...
5       This is an increase of     cents over the tax...
6       Property Tax revenues  including payments and...
7       In addition  the County expects an increase i...
8       Recent state   legislation tied to Medicaid r...
9       As a result  sales tax revenues are expected ...
10      Additional details regarding the county s rev...
11      FY           Proposed Revenues Change from Pr...
12      Millions    Total Assessed Property Values Al...
13      Property tax revenues are based on the assess...
14      For FY              the ad valorem property t...
15                                 Current year taxes  i
16                                                     e
17       taxes paid on time  wi

In [16]:
df = df[:249]

In [17]:
df.head()

,text
0,...
1,This is million higher than the...
2,Property Tax revenues will account for ...
3,The next largest sources of funds for gener...
4,The FY adopted budget includes a gene...


In [18]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [19]:
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [20]:
df.head()

,text
0,...
1,This is million higher than the...
2,Property Tax revenues will account for ...
3,The next largest sources of funds for gener...
4,The FY adopted budget includes a gene...


In [21]:
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [22]:
df2 = df[df['text'].map(lambda d: len(d)) > 0]

In [23]:
!pip install afinn

In [24]:
from afinn import Afinn

In [25]:
afinn = Afinn(language='en')

In [26]:
df['afinn_score'] = df['text'].apply(afinn.score)

In [27]:
df.head()

,text,afinn_score
0,General Fund Revenues Proposed General Fund re...,0.0
1,This million higher than budget approved,2.0
2,Property revenues will account total revenue b...,0.0
3,next largest sources funds general county oper...,0.0
4,adopted budget includes general county wide ra...,0.0


In [28]:
def label_sentiment (score):   
    if score < 0:
        return '0'
    if score == 0:
        return '1'
    if score > 0:
        return '1'

In [29]:
df['emotion'] = df['afinn_score'].apply(label_sentiment)